# Project Titan - Notebook 1: Data Engineering & CRSP-Compustat Merge

### **Objective**
This notebook constructs the foundational dataset for our quantitative analysis. It takes the pre-filtered, large-scale datasets from CRSP and Compustat (prepared offline in Stata for efficiency) and performs the crucial **CRSP-Compustat Merge (CCM)**. The goal is to produce a single, clean, "point-in-time" monthly panel dataset that correctly aligns market data (like returns) with the appropriate lagged fundamental data from financial statements.

---

### **Methodology: The Point-in-Time Merge**

The core challenge in building a research-quality dataset is the correct temporal alignment of data from different sources. Market data (CRSP) is available daily, while fundamental data (Compustat) is reported quarterly with a significant lag. A naive merge would introduce severe **lookahead bias**.

The methodology applied here follows the standard academic and professional approach:

*   **1. Link File Preparation:** First load the clean CCM linking table, which provides the historical mapping between CRSP's `PERMNO` and Compustat's `GVKEY`.

*   **2. Event-Driven Merge (`merge_asof`):** We use Python's `pandas` library, specifically the powerful `pd.merge_asof()` function. This function performs a "nearest-in-time" merge, which is the correct tool for this problem. The process is a two-step merge:
    *   First, we merge the CRSP market data with the CCM linking table to assign the correct `GVKEY` to each `PERMNO` for each point in time.
    *   Second, we merge this combined market data with the Compustat fundamental data. The `merge_asof` ensures that for any given month's market data, we are merging it with the **most recently available public financial statement data**, correctly simulating the information delay.

*   **3. Feature Engineering & Finalization:** Once the panel is constructed, we calculate key derived variables (e.g., market capitalization, excess returns) and resample the data to a monthly frequency.

---
**Output:** This notebook's final output is a single, analysis-ready `panel_data.parquet` file. Parquet format is highly efficient for storing large, structured datasets, preserving data types and offering significant speed advantages over CSV.


In [ ]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

onedrive_root = str(Path(os.environ['OneDrive']))
INPUT_DATA_DIR = os.path.join(onedrive_root, "0. DATASETS", "temps")

# --- Define File Paths ---
CRSP_FILE = os.path.join(INPUT_DATA_DIR, 'crsp_clean_daily.dta')
COMP_FILE = os.path.join(INPUT_DATA_DIR, 'compustat_clean_quarterly.dta')
CCM_FILE = os.path.join(INPUT_DATA_DIR, 'ccm_linking_table_clean.dta')
OUTPUT_FILE = os.path.join('data', 'panel_data.parquet')

print("Setup complete.")


Setup complete.


### 2. Load the Three Cleaned CSV Files

In [5]:
# Load crsp data 
crsp = pd.read_stata(CRSP_FILE)
crsp.rename(columns= {'ret' : 'ret_daily'}, inplace = True) 
print("\n Converting date columns in crsp to datetime objects...")
crsp['date'] = pd.to_datetime(crsp['date'],  format = '%d%b%Y')

# Load Compustat Data
comp = pd.read_stata(COMP_FILE)

# Load CCM Linking Table
ccm = pd.read_stata(CCM_FILE)

print("CRSP, Compustat, and CCM data loaded successfully.")
print(f"CRSP shape: {crsp.shape}")
print(f"Compustat shape: {comp.shape}")



 Converting date columns in crsp to datetime objects...
CRSP, Compustat, and CCM data loaded successfully.
CRSP shape: (58457885, 7)
Compustat shape: (605175, 23)


In [6]:
crsp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58457885 entries, 0 to 58457884
Data columns (total 7 columns):
 #   Column     Dtype         
---  ------     -----         
 0   permno     int32         
 1   hsiccd     float64       
 2   date       datetime64[ns]
 3   prc        float32       
 4   vol        float64       
 5   ret_daily  float32       
 6   shrout     float64       
dtypes: datetime64[ns](1), float32(2), float64(3), int32(1)
memory usage: 2.4 GB


In [7]:
print(crsp['permno'].dtype, crsp['date'].dtype)
print(ccm['permno'].dtype, ccm['link_start_date'].dtype)

int32 datetime64[ns]
int32 datetime64[ns]


### 3. The CRSP-CCM Merge

In [8]:
# --- Step 1 of the Merge: Link CRSP and CCM ---

# Sort both tables by the linking key (permno) and the date
crsp.sort_values(by=['date', 'permno'], inplace=True)
ccm.sort_values(by=['link_start_date', 'permno' ], inplace=True)

# After sorting, we reset the index. This ensures the DataFrame's internal
# structure is clean and satisfies the strict sorting requirement of merge_asof.
crsp.reset_index(drop=True, inplace=True)
ccm.reset_index(drop=True, inplace=True)

# nearest merge to crsp going backwards in time
# Perform the backward merge_asof to find the correct link for each CRSP observation
# merge_asof: for each row in left (crsp date), find the nearest earlier matching row in right (ccm link start).
# This finds the most recent link that was active as of the CRSP date.
# => looks in ccm for the most recent link_start_date that is ≤  CRSP date, and attaches that row.

crsp_ccm = pd.merge_asof(
    left=crsp,
    right=ccm,
    left_on='date',
    right_on='link_start_date',
    by='permno'
)

# the above matches each crsp date to all link_start_dates. some of the linked matches ended prior to cris date. 
# => filter out any matches where the link was no longer valid
# i.e., where the CRSP date is after the link's end date
crsp_ccm = crsp_ccm[crsp_ccm['date'] <= crsp_ccm['link_end_date']]

# calculate market capitalization as number of shares outstanding x share price
crsp_ccm['mkt_cap'] = crsp_ccm['prc'] * crsp_ccm['shrout']
print("CRSP and CCM merged successfully.")

CRSP and CCM merged successfully.


### 4. Merge with Compustat

In [9]:
# sort by date and the linking key (gvkey)
crsp_ccm.sort_values(by = ['date', 'gvkey'], inplace = True)
comp.sort_values(by = ['datadate', 'gvkey'], inplace = True)

panel_data = pd.merge_asof(left = crsp_ccm, 
                        right = comp, 
                        left_on = 'date', 
                        right_on = 'datadate', 
                        by = 'gvkey')

# drop the helper columns
panel_data.drop(columns=['link_start_date', 'link_end_date'], inplace=True)

print("Full daily panel data constructed.")
print(f"Daily panel shape: {panel_data.shape}")

Full daily panel data constructed.
Daily panel shape: (48965243, 31)


In [10]:
panel_data_prev = panel_data.iloc[:10000,:]
panel_data_prev

,permno,hsiccd,date,prc,vol,ret_daily,shrout,gvkey,mkt_cap,datadate,...,ibq,lctq,ltq,oiadpq,pstkq,saleq,oancfy,dvpspq,prccq,sic
0,54594,3724.0,1995-01-03,13.125,13600.0,-0.018692,15907.0,001004,208779.375,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10074,3460.0,1995-01-03,11.750,2045.0,0.000000,5052.0,001009,59361.000,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10082,4840.0,1995-01-03,9.000,8800.0,0.028571,10425.0,001011,93825.000,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,50906,3661.0,1995-01-03,48.750,134350.0,-0.025000,27888.0,001013,1359540.000,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10154,3662.0,1995-01-03,9.250,14000.0,0.000000,3350.0,001017,30987.500,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,75083,2752.0,1995-01-04,10.000,1600.0,0.012658,17375.0,008293,173750.000,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9996,60899,4812.0,1995-01-04,3.000,9900.0,0.000000,7102.0,008296,21306.000,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9997,54463,6711.0,1995-01-04,14.875,242700.0,-0.008333,75510.0,008299,1123211.250,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9998,35051,3569.0,1995-01-04,19.000,156800.0,0.000000,115324.0,008304,2191156.000,1994-10-31,...,13.309,242.177,359.358,19.824,0.0,159.195,48.422,0.185,18.125,3569


### 5. Resample to Monthly and Calculate Monthly Returns


In [11]:
# We will group by each firm (permno) and then by month
# and take the last observation of the month.
# This gives us month-end values for market cap, fundamentals, etc.

monthly_panel = panel_data.groupby('permno').resample('ME', on = 'date').last()


In [12]:
monthly_panel.head()

permno  hsiccd       prc    vol  ret_daily  shrout  \
permno date                                                              
10001  1995-01-31  10001.0  4925.0  7.750000    0.0   0.026915  2224.0   
       1995-02-28  10001.0  4925.0  7.546875  400.0  -0.026210  2224.0   
       1995-03-31  10001.0  4925.0  7.500000  200.0  -0.032258  2244.0   
       1995-04-30  10001.0  4925.0  7.500000  600.0  -0.006211  2244.0   
       1995-05-31  10001.0  4925.0  7.875000    0.0   0.000000  2244.0   

                    gvkey   mkt_cap   datadate  fyearq  ...    ibq   lctq  \
permno date                                             ...                 
10001  1995-01-31  012994  17236.00 1994-12-31  1995.0  ...  0.802  8.520   
       1995-02-28  012994  16784.25 1994-12-31  1995.0  ...  0.802  8.520   
       1995-03-31  012994  16830.00 1995-03-31  1995.0  ...  1.045  6.108   
       1995-04-30  012994  16830.00 1995-03-31  1995.0  ...  1.045  6.108   
       1995-05-31  012994  17671.50 1995-03-31  1995.0  ...  1.045  6.108   

                      ltq  oiadpq  pstkq   saleq  oancfy  dvpspq  prccq   sic  
permno date                                                                    
10001  1995-01-31  23.217   1.486    0.0  10.537     NaN    0.00    8.0  4924  
       1995-02-28  23.217   1.486    0.0  10.537     NaN    0.00    8.0  4924  
       1995-03-31  20.823   1.829    0.0  11.266     NaN    0.19    7.5  4924  
       1995-04-30  20.823   1.829    0.0  11.266     NaN    0.19    7.5  4924  
       1995-05-31  20.823   1.829    0.0  11.266     NaN    0.19    7.5  4924  

[5 rows x 30 columns]

In [13]:
# Dropping the extra 'permno' column from the index
monthly_panel.reset_index(level = 0 , drop = True, inplace = True)

In [14]:
# Calculate monthly returns by compounding the daily returns within each month
# 1+r_month = (1+r_1)(1+r_2)...(1+r_30)
monthly_returns = panel_data.groupby(['permno', pd.Grouper(key='date', freq='ME')])['ret_daily'].apply(lambda x: (1 + x).prod() - 1)
# turning it to dataframe and neming the ret_monthly column 
monthly_returns = monthly_returns.to_frame(name='ret_monthly')

In [15]:
# Merge the monthly returns back into our main panel
# join takes column from the left df and pairs with index of the right df 
monthly_panel = monthly_panel.join(monthly_returns, on=['permno', 'date'])

In [16]:
monthly_panel.head()

,permno,hsiccd,prc,vol,ret_daily,shrout,gvkey,mkt_cap,datadate,fyearq,...,lctq,ltq,oiadpq,pstkq,saleq,oancfy,dvpspq,prccq,sic,ret_monthly
date,,,,,,,,,,,,,,,,,,,,,
1995-01-31,10001.0,4925.0,7.750000,0.0,0.026915,2224.0,012994,17236.00,1994-12-31,1995.0,...,8.520,23.217,1.486,0.0,10.537,NaN,0.00,8.0,4924,-0.03125
1995-02-28,10001.0,4925.0,7.546875,400.0,-0.026210,2224.0,012994,16784.25,1994-12-31,1995.0,...,8.520,23.217,1.486,0.0,10.537,NaN,0.00,8.0,4924,-0.02621
1995-03-31,10001.0,4925.0,7.500000,200.0,-0.032258,2244.0,012994,16830.00,1995-03-31,1995.0,...,6.108,20.823,1.829,0.0,11.266,NaN,0.19,7.5,4924,0.00597
1995-04-30,10001.0,4925.0,7.500000,600.0,-0.006211,2244.0,012994,16830.00,1995-03-31,1995.0,...,6.108,20.823,1.829,0.0,11.266,NaN,0.19,7.5,4924,0.00000
1995-05-31,10001.0,4925.0,7.875000,0.0,0.000000,2244.0,012994,17671.50,1995-03-31,1995.0,...,6.108,20.823,1.829,0.0,11.266,NaN,0.19,7.5,4924,0.05000


In [17]:
# Forward-fill any fundamentals that might be missing for a month
# makea list of fundamentals columns to be ffilled
ffill_cols = comp.columns.drop(['datadate', 'gvkey'])
monthly_panel[ffill_cols] = monthly_panel.groupby('permno')[ffill_cols].ffill()

In [18]:
# any entry with missing market cap or monthly returns will be dropped
monthly_panel.dropna(subset=['ret_monthly', 'mkt_cap'], inplace=True)

In [19]:
monthly_panel.columns

Index(['permno', 'hsiccd', 'prc', 'vol', 'ret_daily', 'shrout', 'gvkey',
       'mkt_cap', 'datadate', 'fyearq', 'fqtr', 'tic', 'actq', 'atq', 'ceqq',
       'cheq', 'cshoq', 'dlcq', 'dlttq', 'dpq', 'ibq', 'lctq', 'ltq', 'oiadpq',
       'pstkq', 'saleq', 'oancfy', 'dvpspq', 'prccq', 'sic', 'ret_monthly'],
      dtype='object')

### 6. Save the Final Panel Datase


In [20]:
# --- Save the Final Dataset in Parquet Format ---

monthly_panel.to_parquet(OUTPUT_FILE)

print(f"Final merged panel data saved to {OUTPUT_FILE}")
print("Notebook 1 (Project Titan) is complete.")

Final merged panel data saved to data\panel_data.parquet
Notebook 1 (Project Titan) is complete.
